<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">DESPLIEGUE DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### DESPLIEGUE POPULARIDAD PELÍCULA.

##### 1. LIBRERÍAS REQUERIDAS Y CONEXIÓN CON GOOGLE DRIVE

In [1]:
#Librerías necesarias para el despliegue.
import pandas as pd
import sqlite3 as sql
import logging
from pathlib import Path
from google.colab import drive

In [2]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


##### 2. DEFINICIÓN DE RUTAS

In [24]:
# Rutas
base_path = Path("/content/drive/MyDrive/marketing/Marketing") # conecta con la carpeta del proyecto
db_file = base_path / "data/db_movies_copy.db" # conecta con la base de datos a usar
output_csv = base_path / "salidas/recomendaciones/peliculas_mas_populares.csv" #crea un archivo csv con la consulta,  la deja en la carpeta recomendaciones, de salidas
output_excel = base_path / "salidas/recomendaciones/peliculas_mas_populares.xlsx" # crea un archivo xlsx con la consulta, la deja en la carpeta recomendaciones, de salidas
log_file = base_path / "salidas/recomendaciones/popular_log.log" # crear el registro de lo que hace se hace en el despligue.


# Crear carpetas si no existen
output_csv.parent.mkdir(parents=True, exist_ok=True)
log_file.parent.mkdir(parents=True, exist_ok=True)

##### 3. CONFIGURACIÓN DEL LOGGING




In [25]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

##### 4. LIMPIAR VARIABLES Y ESPACIO DE TRABAJO.

In [26]:
if logger.hasHandlers():
    logger.handlers.clear()

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler(str(log_file))
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

##### 5. FUNCIONES

In [27]:
# Peliculas más populares por calificación
def obtener_top_peliculas_populares(db_path, top_n=10):
    """
    Obtiene las películas más populares por número de calificaciones.
    """
    logging.info("Inicio del proceso para obtener películas más populares")
    conn = None
    try:
        conn = sql.connect(str(db_path))
        query = f"""
        SELECT m.titulo, COUNT(r.movieId) AS num_ratings
        FROM movies m
        JOIN ratings r ON m.movieId = r.movieId
        GROUP BY m.titulo
        ORDER BY num_ratings DESC
        LIMIT {top_n};
        """
        df = pd.read_sql_query(query, conn)
        logging.info(f"Se recuperaron {len(df)} películas populares exitosamente")
        return df
    except Exception as e:
        logging.error(f"Error al obtener películas populares: {e}")
        print(f"Error al obtener películas populares: {e}")
        return pd.DataFrame()
    finally:
        if conn:
            conn.close()

def guardar_populares_en_csv(df, filepath):
    """
    Guarda el DataFrame en CSV.
    """
    try:
        df.to_csv(filepath, index=False, encoding='utf-8')
        logging.info(f"Películas populares guardadas en CSV: {filepath}")
        print(f"Películas populares guardadas en CSV: {filepath}")
    except Exception as e:
        logging.error(f"Error al guardar en CSV: {e}")
        print(f"Error al guardar en CSV: {e}")

def guardar_populares_en_excel(df, filepath):
    """
    Guarda el DataFrame en Excel.
    """
    try:
        df.to_excel(filepath, index=False, engine='openpyxl')
        logging.info(f"Películas populares guardadas en Excel: {filepath}")
        print(f"Películas populares guardadas en Excel: {filepath}")
    except Exception as e:
        logging.error(f"Error al guardar en Excel: {e}")
        print(f"Error al guardar en Excel: {e}")

##### 5. EJECUCCIÓN DEL DESPLIEGUE.

In [28]:
#Ejemplo popularidad de peliculas.
top_peliculas_df = obtener_top_peliculas_populares(db_file, top_n=10)

if not top_peliculas_df.empty:
    guardar_populares_en_csv(top_peliculas_df, output_csv)
    guardar_populares_en_excel(top_peliculas_df, output_excel)
else:
    print("No se pudieron obtener las películas más populares.")

2025-05-12 22:54:03,739 - INFO - Inicio del proceso para obtener películas más populares
2025-05-12 22:54:04,112 - INFO - Se recuperaron 10 películas populares exitosamente
2025-05-12 22:54:04,122 - INFO - Películas populares guardadas en CSV: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares.csv
2025-05-12 22:54:04,143 - INFO - Películas populares guardadas en Excel: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares.xlsx


Películas populares guardadas en CSV: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares.csv
Películas populares guardadas en Excel: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares.xlsx


#### DEPLIGUE POPULARIDAD POR DÍA DEL PROMEDIO DE CALIFICACIÓN DE LA PELÍCULA.

In [29]:
# Paths
base_path = Path("/content/drive/MyDrive/marketing/Marketing")
db_file = base_path / "data/db_movies_copy.db"
output_csv2 = base_path / "salidas/recomendaciones/peliculas_mas_populares_dia.csv"
output_excel2 = base_path / "salidas/recomendaciones/peliculas_mas_populares_dia.xlsx"
# Día para filtrar
dia = 2

In [32]:
# Funciones
def obtener_top_peliculas_por_dia(db_path, dia, top_n=10):
    try:
        conn = sql.connect(db_path)
        query = f"""
        SELECT r.titulo, r.day, AVG(r.rating) AS avg_rating
        FROM ratings_movies r
        WHERE r.day = {dia}
        GROUP BY r.titulo
        ORDER BY avg_rating DESC
        LIMIT {top_n};
        """
        return pd.read_sql(query, conn)
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

def guardar_en_csv(df, filepath):
    df.to_csv(filepath, index=False)
    print(f"✅ Guardado CSV: {filepath}")

def guardar_en_excel(df, filepath):
    df.to_excel(filepath, index=False, engine='openpyxl')
    print(f"✅ Guardado Excel: {filepath}")

# Ejecución
top_peliculas = obtener_top_peliculas_por_dia(db_file, dia)

if not top_peliculas.empty:
    guardar_en_csv(top_peliculas, output_csv2)
    guardar_en_excel(top_peliculas, output_excel2)
else:
    print("⚠️ No se encontraron películas.")

✅ Guardado CSV: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares_dia.csv
✅ Guardado Excel: /content/drive/MyDrive/marketing/Marketing/salidas/recomendaciones/peliculas_mas_populares_dia.xlsx
